# JellyDB Live Demonstration
### Presented by Team JeLLY-B  

# Performance Testing
Calling __main.py

In [63]:
!python -m JellyDB

from JellyDB.db import Database
from JellyDB.query import Query
from JellyDB.config import Config

Inserting 10k records took:  			 2.25
Updating 10k records took:  			 5.3125
Selecting 10k records took:  			 2.890625
Aggregate 10k of 100 record batch took:	 0.0
Deleting 10k records took:  			 1.8125


# Basic Functionality 

At its core, JellyDB effectively stores data in a simple, L-Store style, columnar storage database. 

**Creating Databases**

By calling 'create_table()' we're able to create a new database based on a user-called key and length.

Suppose we survey individuals for their three favorite numbers: 

In [65]:
db = Database()

fav_numbers = db.create_table('fav_numbers', 4, 0)

try:
    assert fav_numbers._name == "fav_numbers"
    assert fav_numbers._num_columns == 4 + Config.METADATA_COLUMN_COUNT
    assert fav_numbers._key == 0
    print("\nTable 'fav_numbers' successfully created.")

except Exception as exc:
    print("\nCreating table FAILED")
    print(traceback.format_exc())


Table 'fav_numbers' successfully created.


Or if we want to record winning lottery numbers by day:

In [66]:
winning_lotto_numbers = db.create_table('winning_lotto', 7, 0)

try:
    assert winning_lotto_numbers._name == "winning_lotto"
    assert winning_lotto_numbers._num_columns == 7 + Config.METADATA_COLUMN_COUNT
    assert winning_lotto_numbers._key == 0
    print("\nTable 'winning_lotto' successfully created.")

except Exception as exc:
    print("winning_lotto create table FAILED")
    print(traceback.format_exc())


Table 'winning_lotto' successfully created.


We're able to create a space to store that data. 

**Inserting and Referencing Table Information**

Once we've built a table, we can insert new records and call them. For our survey responses, we'll input the response number and each respective set of three responses: 

In [67]:
query = Query(fav_numbers)

response1 = [1, 5, 6, 7]
response2 = [2, 25, 26, 27]
response3 = [3, 99, 98, 97]

query.insert(*response1)
query.insert(*response2)
query.insert(*response3)

try:
    for testkey in [1, 2, 3]:
        s = query.select(testkey, [1, 1, 1, 1])[0].columns
        assert len(s) == 4
        assert s[0] == testkey
    print("\nInserting records into fav_numbers passed.")
except Exception as exc:
        print("\nTable insertions FAILED")
        print(traceback.format_exc())


Inserting records into fav_numbers passed.


Should we (accidentally) try to insert the respondant's information again we'll throw an error: 

In [68]:
query.insert(*response3) # value already exists

Exception: Error: The primary key 3 is already in use

This helps to maintain the integrity of the database. 

We can see that the records were successfully inserted on the first round.


In [69]:
for responses in [1, 2, 3]:
            # Get all columns
            print(query.select(responses, [1, 1, 1, 1])[0].columns)


[1, 5, 6, 7]
[2, 25, 26, 27]
[3, 99, 98, 97]


**Updating Records**

To update a set of responses, however, we can simply call the update function for a specific key. 

If we want to update the third set of responses: 

In [ ]:
query.update(3, *(None, 12, 20, 24))

print("Updated Values: ")
print(query.select(3, [1, 1, 1, 1])[0].columns)

**Deleting Records**

In accordance with good database structure, the user is also able to delete existing records. 

In [70]:
s = query.delete(1)

try: 
    assert s == None
    print("Record successfully deleted.")
except Exception as exc:
    print("\nRecord deletion FAILED")
    print(traceback.format_exc())

Record successfully deleted.


**Summation of Records**

In addition to basic SQL functions, we can also add two values together. In this case, we want to add the second column of rows with primary keys 2 and 3: 

In [80]:
print(query.select(2, [1, 1, 1, 1])[0].columns)
print(query.select(3, [1, 1, 1, 1])[0].columns)

query.sum(2,3,2)

[2, 25, 26, 27]
[3, 99, 98, 97]


124

## Error Handling

While we've shown that we can handle duplication errors on insertion, we've also accounted for several other user errors: 

 - Attempting to delete a non-existent record


In [85]:
# delete column 1.. again. 
query.delete(1)


IndexError: list index out of range

 - Attempting to update a deleted record 

In [84]:
# update deleted column 1
query.update(1, *(None, 12, 20, 34))

IndexError: list index out of range